In [194]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from scipy import stats
import statsmodels.api as sm
from itertools import combinations
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

# **Data cleaning + preparation**


In [195]:
data = pd.read_csv("test_payment_data.csv")

In [196]:
columns_to_fill = ["error_type"]
for col in columns_to_fill:
    data[col].fillna(0, inplace=True)

In [197]:
missing_values_full = data.isnull().sum()
missing_values_full

#                        0
order_id                 0
event_time               0
user_id                  0
price                    0
payment_number           0
transaction_status       0
card_brand               0
card_type              770
bank_name             2009
error_type               0
currency                 0
card_country           527
dtype: int64

In [198]:
data['bank_name'].fillna('Unknown', inplace=True)

In [199]:
data['transaction_status'] = data['transaction_status'].map({'fail': 0, 'success': 1})

In [200]:
data['card_country'].fillna('Unknown', inplace=True)

In [201]:
data['card_type'].fillna('Unknown', inplace=True)

In [202]:
print(data.isnull().sum())

#                     0
order_id              0
event_time            0
user_id               0
price                 0
payment_number        0
transaction_status    0
card_brand            0
card_type             0
bank_name             0
error_type            0
currency              0
card_country          0
dtype: int64


In [203]:
data = data.drop_duplicates(subset=['order_id'], keep='first')

data.to_csv("new_test_payment_data.csv", index=False)

In [204]:
data['event_time'] = pd.to_datetime(data['event_time'])

data_sorted = data.sort_values(by=['user_id', 'event_time'])

data_sorted.to_csv("sorted_by_user_and_time.csv", index=False)

In [205]:
data = pd.read_csv("sorted_by_user_and_time.csv")

user_id_mapping = {uid: i+1 for i, uid in enumerate(data['user_id'].unique())}

data['user_id'] = data['user_id'].map(user_id_mapping)

In [206]:
data = pd.read_csv("sorted_users_with_ids.csv")

summary = data.groupby('user_id').agg(
    total_transactions=('transaction_status', 'count'),
    success_count=('transaction_status', lambda x: (x == 1).sum()),
    fail_count=('transaction_status', lambda x: (x == 0).sum())
).reset_index()


In [207]:
data = pd.read_csv("sorted_users_with_ids.csv")

summary = data.groupby('user_id').agg(
    total_transactions=('transaction_status', 'count'),
    success_count=('transaction_status', lambda x: (x == 1).sum()),
    fail_count=('transaction_status', lambda x: (x == 0).sum())
).reset_index()


summary['success_rate_percent'] = round(
    (summary['success_count'] / summary['total_transactions']) * 100, 2
)

summary.to_csv("user_transaction_summary.csv", index=False)

In [208]:
stats = summary['success_rate_percent'].describe()

variance = summary['success_rate_percent'].var()
coef_variation = summary['success_rate_percent'].std() / summary['success_rate_percent'].mean()

stats['variance'] = variance
stats['coefficient_of_variation'] = coef_variation

print("Статистичні показники success_rate_percent:")
print(stats)


Статистичні показники success_rate_percent:
count                       115245.000000
mean                            36.122848
std                             40.072424
min                              0.000000
25%                              0.000000
50%                             16.670000
75%                             75.000000
max                            100.000000
variance                      1605.799151
coefficient_of_variation         1.109337
Name: success_rate_percent, dtype: float64


# **User reliability**


In [209]:

user_success = data.groupby('user_id').agg(
    total_transactions=('transaction_status', 'count'),
    success_count=('transaction_status', lambda x: (x == 1).sum())
)
user_success['success_rate'] = user_success['success_count'] / user_success['total_transactions']

threshold = 0.6

reliable_users = user_success[user_success['success_rate'] >= threshold]

print(f"Кількість надійних користувачів (поріг {threshold*100}%): {len(reliable_users)}")
print(f"Всього користувачів: {len(user_success)}")
print(f"Частка надійних користувачів: {round(len(reliable_users) / len(user_success) * 100, 2)}%")


Кількість надійних користувачів (поріг 60.0%): 33812
Всього користувачів: 115245
Частка надійних користувачів: 29.34%


In [210]:
mean = user_success['success_rate'].mean()
std = user_success['success_rate'].std()

threshold = mean + std
reliable_users = user_success[user_success['success_rate'] >= threshold]

print(threshold)


0.7619508012717939


In [211]:
user_success['success_rate_percent'] = user_success['success_rate'] * 100

user_success['is_reliable_user'] = (user_success['success_rate_percent'] > 60).astype(int)


print(user_success[['success_rate_percent', 'is_reliable_user']].head())


         success_rate_percent  is_reliable_user
user_id                                        
1                    0.000000                 0
2                   75.000000                 1
3                    0.000000                 0
4                  100.000000                 1
5                   16.666667                 0


In [212]:
summary['is_reliable_user'] = (summary['success_rate_percent'] > 60).astype(int)

summary.to_csv("user_transaction_summary_updated.csv", index=False)


Файл збережено як user_transaction_summary_updated.csv


In [213]:

user_avg_price = data.groupby('user_id')['price'].mean().rename('avg_price')

summary = data.groupby('user_id').agg(
    total_transactions=('transaction_status', 'count'),
    success_count=('transaction_status', lambda x: (x == 1).sum())
)
summary['success_rate_percent'] = (summary['success_count'] / summary['total_transactions']) * 100

summary = summary.join(user_avg_price)

price_threshold = 25
summary['high_avg_price'] = (summary['avg_price'] > price_threshold).astype(int)

print(summary[['success_rate_percent', 'avg_price', 'high_avg_price']].head())


         success_rate_percent  avg_price  high_avg_price
user_id                                                 
1                    0.000000   5.000000               0
2                   75.000000  46.500000               1
3                    0.000000   5.000000               0
4                  100.000000  60.333333               1
5                   16.666667  26.833333               1


In [214]:
successful_only = data[data['transaction_status'] == 1]

user_avg_price = successful_only.groupby('user_id')['price'].mean().rename('avg_price')

summary = data.groupby('user_id').agg(
    total_transactions=('transaction_status', 'count'),
    success_count=('transaction_status', lambda x: (x == 1).sum())
)
summary['success_rate_percent'] = (summary['success_count'] / summary['total_transactions']) * 100

summary = summary.join(user_avg_price)

price_threshold = 25
summary['high_avg_price'] = (summary['avg_price'] > price_threshold).astype(int)

print(summary[['success_rate_percent', 'avg_price', 'high_avg_price']].head())

         success_rate_percent  avg_price  high_avg_price
user_id                                                 
1                    0.000000        NaN               0
2                   75.000000  53.000000               1
3                    0.000000        NaN               0
4                  100.000000  60.333333               1
5                   16.666667   5.000000               0


In [215]:
overall_avg_price = summary['avg_price'].mean()

print(f"Середній середній чек: {round(overall_avg_price, 2)} USD")


Середній середній чек: 16.39 USD


In [216]:
mean = summary['avg_price'].mean()
std = summary['avg_price'].std()
price_threshold = mean + std

price_threshold

29.158604726504095

In [217]:
summary['avg_price'].median()

13.5

In [218]:
summary['avg_price'].describe()


count    70181.000000
mean        16.391333
std         12.767272
min          4.000000
25%          5.000000
50%         13.500000
75%         25.200000
max         73.000000
Name: avg_price, dtype: float64

In [219]:
successful_only = data[data['transaction_status'] == 1]
user_avg_price = successful_only.groupby('user_id')['price'].mean().rename('avg_price')

summary = data.groupby('user_id').agg(
    total_transactions=('transaction_status', 'count'),
    success_count=('transaction_status', lambda x: (x == 1).sum())
)
summary = summary.join(user_avg_price)

price_threshold = 13.5
high_avg_price_users = summary[summary['avg_price'] > price_threshold]

print(f"Кількість платоспроможних користувачів (avg_price > {price_threshold}): {len(high_avg_price_users)}")
print(f"Всього користувачів: {len(summary)}")
print(f"Частка платоспроможних користувачів: {round(len(high_avg_price_users) / len(summary) * 100, 2)}%")


Кількість платоспроможних користувачів (avg_price > 13.5): 35087
Всього користувачів: 115245
Частка платоспроможних користувачів: 30.45%


In [221]:
successful_only = data[data['transaction_status'] == 1]
user_avg_price = successful_only.groupby('user_id')['price'].mean().rename('avg_price')

summary = data.groupby('user_id').agg(
    total_transactions=('transaction_status', 'count'),
    success_count=('transaction_status', lambda x: (x == 1).sum())
)
summary['success_rate'] = summary['success_count'] / summary['total_transactions']


summary = summary.join(user_avg_price)


price_threshold = 13.5
rate_threshold = 0.6

filtered_users = summary[
    (summary['avg_price'] > price_threshold) &
    (summary['success_rate'] >= rate_threshold)
]
filtered_users = summary[
    (summary['success_rate'] >= rate_threshold)
]


total_users = len(summary)
selected_users = len(filtered_users)
selected_percent = round(selected_users / total_users * 100, 2)


print(f"Кількість користувачів, які пройшли обидва фільтри: {selected_users}")
print(f"Всього користувачів: {total_users}")
print(f"Частка таких користувачів від загального числа: {selected_percent}%")


Кількість користувачів, які пройшли обидва фільтри: 33812
Всього користувачів: 115245
Частка таких користувачів від загального числа: 29.34%


In [222]:
data = pd.read_csv('new_test_payment_data.csv')

successful_only = data[data['transaction_status'] == 1]
user_avg_price = successful_only.groupby('user_id')['price'].mean().rename('avg_price')


summary = data.groupby('user_id').agg(
    total_transactions=('transaction_status', 'count'),
    success_count=('transaction_status', lambda x: (x == 1).sum())
)
summary['success_rate'] = summary['success_count'] / summary['total_transactions']

summary = summary.join(user_avg_price)

price_threshold = 13.5
rate_threshold = 0.6
summary['is_reliable_user'] = (
    (summary['avg_price'] > price_threshold) &
    (summary['success_rate'] >= rate_threshold)
).astype(int)


data = data.merge(summary['is_reliable_user'], on='user_id', how='left')


data.to_csv('payment_data_with_reliability.csv', index=False)


In [223]:
multi_tx_df = pd.read_csv("sorted_users_with_ids.csv")
data = pd.read_csv("new_test_payment_data.csv")

successful_only = data[data['transaction_status'] == 1]
user_avg_price = successful_only.groupby('user_id')['price'].mean().rename('avg_price')


summary = data.groupby('user_id').agg(
    total_transactions=('transaction_status', 'count'),
    success_count=('transaction_status', lambda x: (x == 1).sum())
)
summary['success_rate'] = summary['success_count'] / summary['total_transactions']
summary = summary.join(user_avg_price)


summary['is_reliable_user'] = (
    (summary['avg_price'] > 13.5) &
    (summary['success_rate'] >= 0.6)
).astype(int)


multi_tx_df['user_id'] = multi_tx_df['user_id'].astype(str)
summary.index = summary.index.astype(str)

multi_tx_df = multi_tx_df.merge(summary['is_reliable_user'], on='user_id', how='left')

multi_tx_df.to_csv("sorted_users_with_reliability.csv", index=False)


In [224]:
summary_df = pd.read_csv("user_transaction_summary_updated.csv")
transactions_df = pd.read_csv("sorted_users_with_ids.csv")


summary_df['user_id'] = summary_df['user_id'].astype(str)
transactions_df['user_id'] = transactions_df['user_id'].astype(str)


merged_df = transactions_df.merge(
    summary_df[['user_id', 'is_reliable_user']],
    on='user_id',
    how='left'
)


merged_df.to_csv("sorted_users_with_reliability.csv", index=False)


$$
\text{logit}(P(\text{надійний})) =
\beta_0 +
\beta_1 \cdot \text{price} +
\beta_2 \cdot \text{payment\_initial} +
\beta_3 \cdot \text{transaction\_status} +
\sum_{i=1}^{n} \beta_{4i} \cdot \text{card\_brand}_i +
\sum_{j=1}^{m} \beta_{5j} \cdot \text{card\_type}_j +
\sum_{k=1}^{p} \beta_{6k} \cdot \text{bank\_name}_k +
\beta_7 \cdot \text{avg\_error\_type} +
\sum_{l=1}^{r} \beta_{8l} \cdot \text{currency}_l +
\sum_{t=1}^{s} \beta_{9t} \cdot \text{card\_country}_t
$$
